## Imports

In [47]:
# data manipulation
import pandas as pd 
import numpy as np 

# plotting
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
plt.rcParams['figure.figsize'] = (20, 10)
plt.style.use('fivethirtyeight')

# fetching
import pandas_datareader as web
import yfinance as yf
yf.pdr_override()
from datetime import datetime, date
import _datetime

# web scrapping
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
warnings.warn('deprecated', DeprecationWarning)

# report writing
from fpdf import FPDF
from PyPDF2 import PdfFileMerger, PdfFileReader
from PIL import Image, ImageOps
import dataframe_image as dfi
import webbrowser
import os

# interactivity 
import ipywidgets as widgets
from IPython.display import display, clear_output

# progress bar
from tqdm.notebook import tqdm

# ssl issues
import ssl 
try:
    _create_unverified_https_context = ssl._create_unverified_context 
except AttributeError: 
    pass 
else: 
    ssl._create_default_https_context = _create_unverified_https_context

## Scraping

In [48]:
def NASDAQ100_symbols_names():
    url = "https://en.wikipedia.org/wiki/Nasdaq-100"
    tables = pd.read_html(url)
    nasdaq_100 = tables[4]
    company_codes = nasdaq_100['Symbol'].dropna().tolist()
    company_names = nasdaq_100['Company'].dropna().tolist()
    return company_codes[:13], company_names[:13]

In [49]:
def CAC40_symbols_names():
    url = "https://fr.wikipedia.org/wiki/CAC_40"
    tables = pd.read_html(url)
    cac_40 = tables[2]
    company_codes = cac_40['Mnémo'].dropna().tolist()
    company_names = cac_40['Société'].dropna().tolist()
    return company_codes[:13], company_names[:13]

In [50]:
def LSE_symbols_names():
    url = "https://en.wikipedia.org/wiki/FTSE_100_Index"
    tables = pd.read_html(url)
    ftse_100 = tables[4]
    company_codes = ftse_100['Ticker'].dropna().tolist()
    company_names = ftse_100['Company'].dropna().tolist()
    return company_codes[:13], company_names[:13]

## Technical Analysis Indicators

In [51]:
def compute_analytics(df, symbol, params):
    STV = params['STV']
    LTV = params['LTV']
    df['PSAR'] = 0.0
    df['EP'] = 0.0
    df['EP-PSAR'] = 0.0
    df['AccFactor'] = 0.0
    df['EP-PSAR*AccFactor'] = 0.0
    df['PSAR_signal'] = ''
    # first, create the derived value 
    df['PSAR'][0] = df['Low'][0]
    df['EP'][0] = df['High'][0]
    df['EP-PSAR'][0] = df['EP'][0] - df['PSAR'][0]
    df['AccFactor'][0] = 0.02
    df['EP-PSAR*AccFactor'][0] = df['EP-PSAR'][0] * df['AccFactor'][0]
    df['PSAR_signal'][0] = 'Bull' if (df['EP-PSAR'][0] >= 0.0) else 'bear'
    #df.drop(['Adj Close'], axis=1)
    df['symbol'] = symbol
    df['upper_band'] = df['Close'].rolling(STV).mean() + df['Close'].rolling(STV).std()*1.1
    df['lower_band'] = df['Close'].rolling(STV).mean() - df['Close'].rolling(STV).std()*1.1
    df['STMAV'] = df['Close'].rolling(STV).mean()
    df['LTMAV'] = df['Close'].rolling(LTV).mean()
    df['n_high'] = df['High'].rolling(LTV).max()
    df['n_low'] = df['Low'].rolling(LTV).min()
    df['%K'] = (df['Close'] - df['n_low']) * 100 / (df['n_high'] - df['n_low'])
    df['%D'] = df['%K'].rolling(5).mean()
    df['delta'] = df['Close'].diff(1)
    df['delta'].dropna(inplace=True)
    df['positive'] = df['delta'].copy()
    df['negative'] = df['delta'].copy()
    df['positive'][df['positive'] < 0] = 0
    df['negative'][df['negative'] > 0] = 0
    df['average_gain'] = df['positive'].rolling(STV).mean()
    df['average_lost'] = abs(df['negative'].rolling(STV).mean())
    df['relative_strength'] = df['average_gain']/df['average_lost']
    df['RSI'] = 100 - (100/ (1+df['relative_strength']))
    return df

In [52]:
def PSAR_calculation(df):
    for i in range(1, len(df)):
        # PSAR calculation
        if (df['PSAR_signal'][i-1] == 'Bull') and (df['PSAR'][i-1]+df['EP-PSAR*AccFactor'][i-1] > df['Low'][i]):
            df['PSAR'][i] = df['EP'][i-1] 
        elif (df['PSAR_signal'][i-1] == 'Bear') and (df['PSAR'][i-1]+df['EP-PSAR*AccFactor'][i-1] < df['High'][i]):
            df['PSAR'][i] = df['EP'][i-1]
        else:
            df['PSAR'][i] = df['PSAR'][i-1]+df['EP-PSAR*AccFactor'][i-1]

        # signal definition
        if df['PSAR'][i] < df['High'][i]:
            df['PSAR_signal'][i] = 'Bull'
        elif df['PSAR'][i] > df['High'][i]:
            df['PSAR_signal'][i] = 'Bear'

        # EP calculation     
        if df['PSAR_signal'][i] == 'Bull' and df['High'][i] > df['EP'][i-1]:
            df['EP'][i] = df['High'][i]
        elif df['PSAR_signal'][i] == 'Bull' and df['High'][i] < df['EP'][i-1]:
            df['EP'][i] = df['EP'][i-1]
        else:
            df['EP'][i] = df['High'][i]

        if df['PSAR_signal'][i] == 'Bear' and df['Low'][i] < df['EP'][i-1]:
            df['EP'][i] = df['Low'][i]
        elif df['PSAR_signal'][i] == 'Bear' and df['Low'][i] > df['EP'][i-1]:
            df['EP'][i] = df['EP'][i-1]
        else:
            df['EP'][i] = df['High'][i]

        # EP-PSAR calculation
        df['EP-PSAR'][i] = df['EP'][i] - df['PSAR'][i]

        # accumulation factor calculation
        if df['EP'][i] != df['EP'][i-1] and df['AccFactor'][i-1]<0.19: 
            df['AccFactor'][i] = df['AccFactor'][i-1] + 0.02
        elif df['PSAR_signal'][i] != df['PSAR_signal'][i-1]:
            df['AccFactor'][i] = 0.02
        else: 
            df['AccFactor'][i] = df['AccFactor'][i-1]

        # EP-PSAR*AccFactor calculation
        df['EP-PSAR*AccFactor'][i] = df['EP-PSAR'][i] * df['AccFactor'][i]
    return df

## Signal trading strategy

In [53]:
def run_analysis(df_in): 
    pdf = None
    summary_table = pd.DataFrame({'Indicators': ['bollinger_bands', 'moving_average', 'stochastic_oscillator', 
                                                 'relative_strength', 'parabolic_sar', 'total']})
    summary_table.set_index('Indicators', inplace=True)
    
    for symbol in df_in['symbol'].unique(): 
        df = df_in[df_in['symbol']==symbol]
        
        bb_signal = False
        maco_signal = False
        so_signal = False
        rsi_signal = False
        psar_signal = False
        
        # bollinger bands signal
        # the signal triggers when 

        if ((df['Close'][-2] < df['upper_band'][-2]) and (df['Close'][-1] > df['upper_band'][-1])) or \
        ((df['Close'][-2] > df['lower_band'][-2]) and (df['Close'][-1] < df['lower_band'][-1])):
            print(f'Bollinger signal was emitted by {symbol}')
            bb_signal=True

        # moving average crossover signal
        #the signal triggers when short moving average crosses from above or below the long term moving average

        if ((df['STMAV'][-3] > df['LTMAV'][-3]) and (df['STMAV'][-1] < df['LTMAV'][-1])) or \
        ((df['STMAV'][-3] < df['LTMAV'][-3]) and (df['STMAV'][-1] > df['LTMAV'][-1])):
            print(f'Moving average crossover signal was emitted by {symbol}')
            maco_signal=True
            
        # stochastic oscillator signal
        if ((df['%K'][-1] > 80) and (df['%D'][-1] > 80)) or \
        ((df['%K'][-1] < 20) and (df['%D'][-1] < 20)): 
            print(f'Stochastic oscillator signal was emitted by {symbol}')
            so_signal=True
        
        # rsi signal
        if (df['RSI'][-1] > 80) or (df['RSI'][-1] < 20): 
            print(f'Relative strengh indicator signal was emitted by {symbol}')
            rsi_signal=True
        
        # parabolic SAR signal
        if (df['PSAR'][-2] > df['Close'][-2] and df['PSAR'][-1] < df['Close'][-1]) or \
        (df['PSAR'][-2] < df['Close'][-2] and df['PSAR'][-1] > df['Close'][-1]): 
            print(f'Parabolic SAR signal was emitted by {symbol}')
            psar_signal = True
 
        # total signals per stock symbol
        signals = (so_signal + maco_signal + bb_signal + rsi_signal + psar_signal)
        
        # build summary table
        if signals >= 2:
            summary_table[f'{symbol}'] = [0, 0, 0, 0, 0, 0]

            if bb_signal:
                summary_table[f'{symbol}'].loc['bollinger_bands'] = 1
            if maco_signal:
                summary_table[f'{symbol}'].loc['moving_average'] = 1
            if so_signal:
                summary_table[f'{symbol}'].loc['stochastic_oscillator'] = 1
            if rsi_signal:
                summary_table[f'{symbol}'].loc['relative_strength'] = 1
            if psar_signal:
                summary_table[f'{symbol}'].loc['parabolic_sar'] = 1
            summary_table[f'{symbol}'].loc['total'] = signals
        
        # draw all plots & generate report when at least 3 signals trigger
        if signals >= 3:
            plot_stoch(f'{symbol}', df['Close'], df['%K'], df['%D'])
            plot_maco(f'{symbol}', df['Close'], df['STMAV'], df['LTMAV'])
            plot_bb(f'{symbol}', df['Close'], df['upper_band'], df['lower_band'])
            plot_rsi(f'{symbol}', df.index, df['Close'], df['RSI'])
            plot_psar(f'{symbol}', df.index, df['Close'], df['PSAR'])
            # generate report
            pdf = generate_pdf_report(pdf, symbol)
            # delete pngs
            os.remove(f'{symbol}SO.png')
            os.remove(f'{symbol}BB.png')
            os.remove(f'{symbol}MACO.png')
            os.remove(f'{symbol}RSI.png')
            os.remove(f'{symbol}PSAR.png')
            os.remove(f'{symbol}SO_border.png')
            os.remove(f'{symbol}BB_border.png')
            os.remove(f'{symbol}MACO_border.png')
            os.remove(f'{symbol}RSI_border.png')
            os.remove(f'{symbol}PSAR_border.png')
    
    summary_table['Total'] = summary_table.sum(axis=1)
    summary_table = summary_table.sort_values(by='total', axis=1, ascending=False)
    if pdf is not None:
        # export dataframe as an image
        dfi.export(summary_table, "mytable.png")
        first_page = first_page_report(summary_table)

        # save summary table and plots
        pdf.output('corps_temp.pdf')
        first_page.output('first_page_temp.pdf')

        # merge summary table and plots
        merger = PdfFileMerger()
        merger.append(PdfFileReader(open('first_page_temp.pdf', 'rb')))
        merger.append(PdfFileReader(open('corps_temp.pdf', 'rb')))
        merger.write("pdf_1.pdf")

    webbrowser.open('file:///Users/albert/Pythonproject/FirstProject/Github%20projects/pdf_1.pdf', new=1)  
    return summary_table

## Plotting

In [54]:
# function that plots stochastic oscillator
def plot_stoch(symbol, price, k, d):
    plt.figure(figsize=(20,15))
    ax1 = plt.subplot2grid((9, 1), (0,0), rowspan = 5, colspan = 1)
    ax2 = plt.subplot2grid((9, 1), (6,0), rowspan = 3, colspan = 1, sharex=ax1)
    ax1.plot(price, linewidth=3)
    ax1.set_title(f'{symbol} stock price')
    ax2.plot(k, linewidth = 3, label = '%K')
    ax2.plot(d, color = 'red', linewidth = 1.5, label = '%D')
    ax2.axhline(80, color = 'black', linewidth = 1.5, linestyle = '--')
    ax2.axhline(20, color = 'black', linewidth = 1.5, linestyle = '--')
    ax2.set_title(f'{symbol} stochastic oscillator')
    plt.savefig(f'{symbol}SO.png', bbox_inches='tight')
    plt.show()

In [55]:
# function that plots relative strength index
def plot_rsi(symbol,index, price, RSI):
    plt.figure(figsize=(20,15))
    ax1 = plt.subplot2grid((9, 1), (0,0), rowspan = 5, colspan = 1)
    ax1.plot(index, price, linewidth=3)
    ax1.set_title(f'{symbol} stock price')
    ax1.grid('--')
    ax1.set_axisbelow(True)
    ax1.tick_params(axis='x')
    ax1.tick_params(axis='y')
    #ax2 = plt.subplot(212, sharex=ax1)
    ax2 = plt.subplot2grid((9, 1), (6,0), rowspan = 3, colspan = 1, sharex=ax1)
    ax2.plot(index, RSI, linewidth=3)
    ax2.set_title(f'{symbol} relative strength index')
    ax2.axhline(20, linestyle='--', color='black', linewidth=1.5)
    ax2.axhline(80, linestyle='--', color='black', linewidth=1.5)
    plt.savefig(f'{symbol}RSI.png', bbox_inches='tight')
    plt.show()

In [56]:
# function that plots bollinger bands
def plot_bb(symbol, price, up, low):
    plt.plot(price, label='closing price', linewidth=3)
    plt.plot(up, '--', label="upper", linewidth=1.5, color='red')
    plt.plot(low, '--', label="lower", linewidth=1.5, color='red')
    plt.title(f'{symbol} bollinger bands')
    plt.savefig(f'{symbol}BB.png', bbox_inches='tight')
    plt.show()

In [57]:
# function that plots moving average crossover 
def plot_maco(symbol, price, ma_short, ma_long):
    plt.plot(price, label='stock price', linewidth=3)
    plt.plot(ma_short, '--', label="upper", linewidth=2)
    plt.plot(ma_long, '--', label="lower", linewidth=2)
    plt.title(f'{symbol} moving average crossover')
    plt.savefig(f'{symbol}MACO.png', bbox_inches='tight')
    plt.show()

In [58]:
# function that plots parabolic stop & reverse
def plot_psar(symbol, index, price, PSAR):
    plt.figure(figsize=(20,10))
    plt.plot(index, price, linewidth=3)
    plt.title(f'{symbol} parabolic sar')
    plt.scatter(index, PSAR, color='red', marker='.')
    plt.savefig(f'{symbol}PSAR.png', bbox_inches='tight')
    plt.show()

## PDF writing

In [59]:
def first_page_report(summary_table):
    pdf = FPDF('P', 'mm', 'Letter')
    
    image = Image.open('mytable.png')
    new_img = ImageOps.expand(image, border=(0, 20, 0, 20), fill='white')
    new_img.save('mytable_border.png')
    image1 = Image.open('mytable_border.png')
    new_img1 = ImageOps.expand(image1, border=(1, 1, 1, 1), fill='black')
    new_img1.save('mytable_border.png')
                  
    pdf.add_page()
    pdf.set_text_color(255, 255, 255)
    pdf.image('Letterhead.png', 0, 0, w = 218)
    pdf.set_font('helvetica', '', 30)
    pdf.cell(0, 10, f'Stock Market Report', ln=1)
    pdf.set_text_color(0, 0, 0)
    pdf.set_font('helvetica', 'B', 17)
    pdf.cell(0, 75, f'Signals summary table for {input_se.value} stock exchange:', ln=1)
    pdf.image('mytable_border.png',25, 75, w=160)
    pdf.image('Letterdown.png', 0, 250, w = 220)
    return pdf

    
def generate_pdf_report(pdf, symbol):
    if not pdf:
        pdf = FPDF('P', 'mm', 'Letter')

    #page 2
    pdf.add_page()
    pdf.image('Letterhead.png', 0, 0, w = 218)
    pdf.set_font('helvetica', '', 20)
    pdf.set_text_color(255, 255, 255)
    pdf.cell(0, 10, f'{symbol} technical analysis', ln=1)
    pdf.ln(20)
    pdf.set_text_color(0, 0, 0)
    pdf.set_font('helvetica', 'B', 15)
    pdf.cell(0, 10, 'Bollinger Bands', ln=1) 
    pdf.ln(5)
    pdf.set_font('helvetica', '', 15)
    pdf.cell(0, 10, '- when the price of the stock is near the lower band, it is considered attractive', ln=1)
    pdf.cell(0, 10, '- if the stock is near the upper band, it is overvalued', ln=1)
    pdf.cell(0, 10, '- if the spread between the closing price and both bands is high, the asset is volatile', ln=1)
    pdf.cell(0, 10, '- if the spread between the closing price and both bands sustain, any existing trend', ln=1)
    pdf.cell(0, 10, 'may be ending', ln=1)
    pdf.cell(0, 10, '- if the volatility is low for 6 months, it is an indication that a period of increased', ln=1)
    pdf.cell(0, 10, 'volatility will follow (bollinger bands squeeze)', ln=1)
    pdf.cell(0, 10, '- a strong trend continuation can be expected when the price moves out of the bands', ln=1)
    pdf.image('Letterdown.png', 0, 250, w = 220)
    image = Image.open(f'{symbol}BB.png')
    new_img = ImageOps.expand(image, border=(1, 1, 1, 1), fill='black')
    new_img.save(f'{symbol}BB_border.png')
    pdf.image(f'{symbol}BB_border.png', 10, 150, w=197)
    
    # page 3
    pdf.add_page()
    pdf.image('Letterhead.png', 0, 0, w = 218)
    pdf.set_font('helvetica', '', 20)
    pdf.set_text_color(255, 255, 255)
    pdf.cell(0, 10, f'{symbol} technical analysis', ln=1)
    pdf.ln(20)
    pdf.set_text_color(0, 0, 0)
    pdf.set_font('helvetica', 'B', 15)
    pdf.cell(0, 10, 'Moving Average Crossovers', ln=1) 
    pdf.ln(5)
    pdf.set_font('helvetica', '', 15)
    pdf.cell(0, 10, '- uses two (or more) moving averages, one slower and one faster', ln=1)
    pdf.cell(0, 10, '- smooths out price by filtering out the noise from random short-term price fluctuations', ln=1)
    pdf.cell(0, 10, '- if the shorter term MA crosses the longer term from below, it is an indication to buy', ln=1)
    pdf.cell(0, 10, '- inversely, if the shorter term crosses the longer term from above, it is an indication', ln=1)
    pdf.cell(0, 10, 'to sell', ln=1)
    pdf.cell(0, 10, '- the shorter is more reactive to daily price changes, as it only considers prices', ln=1)
    pdf.cell(0, 10, 'over short period of time', ln=1)
    pdf.image('Letterdown.png', 0, 250, w = 220)
    
    image = Image.open(f'{symbol}MACO.png')
    new_img = ImageOps.expand(image, border=(1, 1, 1, 1), fill='black')
    new_img.save(f'{symbol}MACO_border.png')
    pdf.image(f'{symbol}MACO_border.png', 10, 150, w=197)
    
    # page 4
    pdf.add_page()
    pdf.image('Letterhead.png', 0, 0, w = 218)
    pdf.set_font('helvetica', '', 20)
    pdf.set_text_color(255, 255, 255)
    pdf.cell(0, 10, f'{symbol} technical analysis', ln=1)
    pdf.ln(20)
    pdf.set_text_color(0, 0, 0)
    pdf.set_font('helvetica', 'B', 15)
    pdf.cell(0, 10, 'Stochastic Oscillator', ln=1) 
    pdf.ln(5)
    pdf.set_font('helvetica', '', 15)
    pdf.cell(0, 10, '- it is a momentum metrics used to predict potential reversals', ln=1) 
    pdf.cell(0, 10, '- the stochastic oscillator is range-bound, between 0 and 100', ln=1) 
    pdf.cell(0, 10, '- when the two lines intersect, we consider that a reversal is to happen', ln=1)
    pdf.cell(0, 10, '- when a bearish trend reaches a new lower low but the oscillator prints a higher low,', ln=1)
    pdf.cell(0, 10, 'it may indicate a bullish reversal', ln=1)
    pdf.image('Letterdown.png', 0, 250, w = 220)
    image = Image.open(f'{symbol}SO.png')
    new_img = ImageOps.expand(image, border=(1, 1, 1, 1), fill='black')
    new_img.save(f'{symbol}SO_border.png')
    pdf.image(f'{symbol}SO_border.png', 10, 110, w=195)

    # page 5
    pdf.add_page()
    pdf.image('Letterhead.png', 0, 0, w = 218)
    pdf.set_font('helvetica', '', 20)
    pdf.set_text_color(255, 255, 255)
    pdf.cell(0, 10, f'{symbol} technical analysis', ln=1)
    pdf.ln(20)
    pdf.set_text_color(0, 0, 0)
    pdf.set_font('helvetica', 'B', 15)
    pdf.cell(0, 10, 'Relative Strength Indicator (RSI)', ln=1) 
    pdf.ln(5)
    pdf.set_font('helvetica', '', 15)
    pdf.cell(0, 10, '- is a tool that measures the magnitude of recent price changes', ln=1) 
    pdf.cell(0, 10, '- typically, values of 80 or above indicate that a security is becoming overbought,', ln=1)
    pdf.cell(0, 10, 'and may be primed for a trend reversal or corrective pullback in price', ln=1)
    pdf.cell(0, 10, '- an RSI reading of 20 or below indicates an oversold or undervalued state', ln=1)
    pdf.cell(0, 10, '- the slope of a momentum oscillator is directly proportional to the velocity of the move', ln=1)
    pdf.image('Letterdown.png', 0, 250, w = 220)
    image = Image.open(f'{symbol}RSI.png')
    new_img = ImageOps.expand(image, border=(1, 1, 1, 1), fill='black')
    new_img.save(f'{symbol}RSI_border.png')
    pdf.image(f'{symbol}RSI_border.png', 10, 110, w=195)
    
    # page 6
    pdf.add_page() 
    pdf.image('Letterhead.png', 0, 0, w = 218)
    pdf.set_font('helvetica', '', 20)
    pdf.set_text_color(255, 255, 255)
    pdf.cell(0, 10, f'{symbol} technical analysis', ln=1)
    pdf.ln(20)
    pdf.set_text_color(0, 0, 0)
    pdf.set_font('helvetica', 'B', 15)
    pdf.cell(0, 10, 'Parabolic Stop & Reverse (PSAR)', ln=1) 
    pdf.ln(5)
    pdf.set_font('helvetica', '', 15)
    pdf.cell(0, 10, '- SAR is also referred to as a Stop And Reverse system (SAR)', ln=1) 
    pdf.cell(0, 10, '- it is a tool that helps predicting market reversal', ln=1)
    pdf.cell(0, 10, '- the direction of the trend using the parabolic SAR should be established 1st', ln=1)
    pdf.cell(0, 10, '- then, other indicators should be used to evaluate strength of movement', ln=1)
    pdf.cell(0, 10, '- if the SAR points are below the closing price, it is a bullish signal', ln=1)
    pdf.cell(0, 10, '- bullish signal: stock will go up', ln=1)
    pdf.cell(0, 10, '- inversely, if the SAR line is above the closing price, it is a bearish signal', ln=1)
    pdf.image('Letterdown.png', 0, 250, w = 220)
    image = Image.open(f'{symbol}PSAR.png')
    new_img = ImageOps.expand(image, border=(1, 1, 1, 1), fill='black')
    new_img.save(f'{symbol}PSAR_border.png')
    pdf.image(f'{symbol}PSAR_border.png', 10, 150, w=197)
    return pdf

## Script execution

In [60]:
def fetch(symbols, company_names, params):
    START = params['START']
    INPUT_SE = params['INPUT_SE']
    STV = params['STV']
    LTV = params['LTV']
    df_concat = pd.DataFrame()
    end = f'{_datetime.date.today()}'
    for idx, symbol in enumerate(tqdm(symbols)):
        try:
            if INPUT_SE == 'NYSE':
                df = round(yf.download(f'{symbol}', start=START, end=end),2)
                print(f'{symbol}')
            elif INPUT_SE == 'FTSE60':
                df = round(yf.download(f'{symbol}.L', start=START, end=end),2)
                print(f'{symbol}')
            elif INPUT_SE == 'CAC40':
                df = round(yf.download(f'{symbol}.PA', start=START, end=end),2)
                print(f'{symbol}')
        
            #print(company_names[idx])
            df = compute_analytics(df, symbol, params)
            
            df = PSAR_calculation(df)
                
            df_concat = pd.concat([df_concat, df])
            
        except Exception:
            pass
    return df_concat

In [61]:
def load_symbols_names(INPUT_SE):
    symbols = []
    company_names = []
    if INPUT_SE == 'CAC40':
        symbols, company_names = CAC40_symbols_names()
    elif INPUT_SE == 'FTSE60':
        symbols, company_names = LSE_symbols_names()
    elif INPUT_SE == 'NYSE':
        symbols, company_names = NASDAQ100_symbols_names()
    return symbols, company_names

In [62]:
def main(event):
    START = f'{date.value}'
    INPUT_SE = input_se.value
    STV = stma.value
    LTV = ltma.value
    params = {'LTV':LTV, 'STV':STV, 'START':START, 'INPUT_SE':INPUT_SE}

    # get list of symbols & company names
    symbols, company_names = load_symbols_names(INPUT_SE)
    # fetch financial data
    df = fetch(symbols, company_names, params)
    # comment
    run_analysis(df)

## Interface and widgets

In [63]:
input_se = widgets.ToggleButtons(options=['FTSE60', 'NYSE', 'CAC40'])

date = widgets.DatePicker()

stma = widgets.IntSlider(min=0,  max=20, step=1)

ltma = widgets.IntSlider(min=0,  max=30, step=1)

In [64]:
# button 

button_summary = widgets.Button(description='Programme summary', tooltip='Click to visualize summary')

button_run_programme = widgets.Button(description='Run the programme')

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        print(f"The programme will scan through the {input_se.value}")
        print(f"It will analyze data starting from {date.value}")
        print(f"The short-term moving value is {stma.value}-days long")
        print(f"and the long-term moving value is {ltma.value}-days long")

button_summary.on_click(on_button_clicked)

button_run_programme.on_click(main)

vbox_result = widgets.VBox([button_summary, output])

vbox2 = widgets.VBox([button_run_programme])

In [65]:
# stacked right hand side

text_0 = widgets.HTML(value="<h3>Stock Market Assistant</h3>")
text_1 = widgets.HTML(value="<h5>pick a stock exchange:</h5>")
text_2= widgets.HTML(value="<h5>pick a start date for the investment time horizon: </h5>")
text_3= widgets.HTML(value="<h5>along with a short-term moving value:</h5>")
text_4= widgets.HTML(value="<h5>and a long-term moving value:</h5>")

vbox_text = widgets.VBox([text_0, text_1, input_se, text_2, date, text_3, 
                          stma, text_4, ltma, vbox_result, vbox2])

In [66]:
page = widgets.HBox([vbox_text])

In [67]:
display(page)